# Stress Signal Classification with Convolutional Neural Networks

**Author:** [Mark Goertz](https://github.com/markgoertz)<br>
**Date created:** 2024/04/05<br>
**Last modified:** 2022/04/05<br>
**Description:** Training a Convolutional model to classify stress signals produced by exposure to certain stimuli.

## Setup and Data Downloads
First, lets install our dependencies:


In [2]:
%pip install os -q
%pip install pandas -q

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import os

C:\Users\goert\AppData\Local\Temp\ipykernel_24724\3026406390.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
MAIN_PATH = os.path.dirname(os.getcwd())
SAVE_PATH = MAIN_PATH + "/data/processed/WESAD"

if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

In [5]:
def read_signal(signal):
    df = pd.read_csv(os.path.join(SAVE_PATH, f"combined_{signal}.csv"), dtype={'id': str})
    return df

In [6]:
eda, temp, label = None, None, None, None
signals = ['w_eda', 'w_temp', 'downsampled_labels']

print("Loading signals..." )

acc = read_signal('acc')
eda = read_signal('eda')
hr = read_signal('hr')
temp = read_signal('temp')

print("Completed phase: 1/3")


Loading signals...
Completed phase: 1/3


In [7]:
def merge(id):
    print(f"Processing {id}")
    columns = ['X', 'Y', 'Z', 'EDA', 'HR', 'TEMP', 'id', 'datetime']
    df = pd.DataFrame(columns=columns)

    acc_id = acc[acc['id'] == id] 
    eda_id = eda[eda['id'] == id].drop(['id'], axis=1)
    hr_id = hr[hr['id'] == id].drop(['id'], axis=1)
    temp_id = temp[temp['id'] == id].drop(['id'], axis=1)

    df = acc_id.merge(eda_id, on='datetime', how='outer')
    df = df.merge(temp_id, on='datetime', how='outer')
    df = df.merge(hr_id, on='datetime', how='outer')

    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)

    return df

In [8]:
# Merge data
print('Merging Data ...')

subjects = eda['id'].unique()
print(subjects)
results = []


for subject in subjects:
    results.append(merge(subject))
new_df = pd.concat(results, ignore_index=True)

print("Completed phase: 2/3")

Merging Data ...
['S2' 'S3' 'S4' 'S5' 'S6' 'S7' 'S8' 'S9' 'S10' 'S11' 'S13' 'S14' 'S15'
 'S16' 'S17']
Processing S2


MergeError: Passing 'suffixes' which cause duplicate columns {'Time_x'} is not allowed.

In [ ]:
print("Saving data ...")
new_df.to_csv(os.path.join(SAVE_PATH, "merged_data.csv"), index=False)

print("Completed phase: 3/3")

Saving data ...
Completed phase: 3/3
